In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset_path = "/content/drive/MyDrive/Datasets/originalimages"

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

def load_images_and_labels(folder, img_size=(100, 100)):
    X, y = [], []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            label = filename.split("-")[0]
            path = os.path.join(folder, filename)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, img_size)
                X.append(img)
                y.append(label)
    return np.array(X), np.array(y)


X, y = load_images_and_labels(dataset_path)


In [ ]:
from skimage.feature import local_binary_pattern

def extract_lbp_features(images, P=8, R=1):
    lbp_features = []
    for img in images:
        lbp = local_binary_pattern(img, P, R, method='uniform')
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, P + 3), range=(0, P + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-7)  # normalize
        lbp_features.append(hist)
    return np.array(lbp_features)

X_lbp = extract_lbp_features(X)


In [ ]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(
    X_lbp, y_cat, test_size=0.2, stratify=y_cat, random_state=42
)


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)  # Adjust as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

mlp_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_pca.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(y_cat.shape[1], activation='softmax')
])

mlp_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = mlp_model.fit(
    X_train_pca, y_train,
    validation_data=(X_test_pca, y_test),
    epochs=25,
    batch_size=32
)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

# Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label="Training Loss")
plt.plot(history.history['val_loss'], label="Validation Loss")
plt.title("Loss Curve")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

# Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label="Training Accuracy")
plt.plot(history.history['val_accuracy'], label="Validation Accuracy")
plt.title("Accuracy Curve")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
def predict_face(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print("❌ Image not found.")
        return
    img = cv2.resize(img, (100, 100))
    lbp = local_binary_pattern(img, P=8, R=1, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 8 + 3), range=(0, 8 + 2))
    hist = hist.astype("float") / (hist.sum() + 1e-7)
    hist_pca = pca.transform([hist])
    prediction = mlp_model.predict(hist_pca)
    predicted_label = np.argmax(prediction)
    identity = le.inverse_transform([predicted_label])[0]

    plt.imshow(img, cmap='gray')
    plt.title(f"Predicted ID: {identity}")
    plt.axis('off')
    plt.show()

    return identity

# Example usage
predict_face("/content/drive/MyDrive/Datasets/originalimages/2-08.jpg")
